# SwissPeaks Data

In this code, Swiss Peaks website is scraped. The raw data is then saved to a folder with race and year. No cleaning is done.


In [17]:
import requests
import json
import html
import pandas as pd
import time
import re
import unicodedata
import datetime
import random
import numpy as np
from datetime import timedelta

# Start the timer
start_time = time.time()

year = '2021'
male_url = 'https://my4.raceresult.com/161079/RRPublish/data/list?key=374250657b42ce1a7a8e1a4a3311786d&listname=Result%20Lists%7CProv.%20Gender%20Results%20360&page=results&contest=1&r=group&name=%232_Male&f='
female_url = 'https://my4.raceresult.com/161079/RRPublish/data/list?key=374250657b42ce1a7a8e1a4a3311786d&listname=Result%20Lists%7CProv.%20Gender%20Results%20360&page=results&contest=1&r=group&name=%231_Female&f='
delay_seconds = 15

## Scraping Swiss Peaks 2021 Data from the website

In [18]:
def reformat_name(name_str):
    parts = [p.strip() for p in name_str.split(',')]
    return f"{parts[1]} {parts[0]}" if len(parts) == 2 else name_str


runner_list = []

for url in [male_url, female_url]:
    print(f"Fetching from: {url}")
    
    # Send GET request
    response = requests.get(url)

    if response.status_code == 200:
        json_data = response.json()
        
        # Extract the specific race data
        race_participants = json_data['data']
    #     print(race_participants)
        extracted_list = []

        for participant in race_participants:
            
            try:
                bib =  participant[0]
                ID =  participant[1]
                status =  participant[2]                
            
                # Format name
                formatted_name = reformat_name(participant[3])
                formatted_name = formatted_name.title()
                
                match = re.search(r'\[img:flags/([A-Z]{2})\.gif\]', participant[4])
                country_code = match.group(1) if match else ""
                
                birth_year =  participant[5]               
                age_gender = participant[6]
                club =  participant[7]       
                
                # Convert time
                raw_time = participant[8]
                formatted_time = pd.to_timedelta(raw_time)
#                 print(formatted_time)    
                
                
                runner_info = [bib, ID, status, formatted_name, 
                               country_code,age_gender, 
                                birth_year, club, formatted_time ]
                print(runner_info, '\n', '*'*20)
                runner_list.append(runner_info)
            except:
                print(participant)


    else:
        print(f"Failed to retrieve data. Status code: {response.status_code}")

    time.sleep(delay_seconds)

Fetching from: https://my4.raceresult.com/161079/RRPublish/data/list?key=374250657b42ce1a7a8e1a4a3311786d&listname=Result%20Lists%7CProv.%20Gender%20Results%20360&page=results&contest=1&r=group&name=%232_Male&f=
['2', '14362', '1.', 'Peter Kienzl', 'IT', 'Master 1 Hommes', '1975', 'Hoka / Dynafit', Timedelta('3 days 14:34:28')] 
 ********************
['4', '12489', '2.', 'Michael Nançoz', 'CH', 'Seniors Hommes', '1988', '', Timedelta('3 days 21:51:22')] 
 ********************
['369', '14181', '3.', 'Hendrik Spoering', 'DE', 'Seniors Hommes', '1990', '', Timedelta('4 days 03:18:46')] 
 ********************
['36', '12514', '4.', 'Nicolas Bie', 'FR', 'Master 1 Hommes', '1977', '', Timedelta('4 days 04:37:23')] 
 ********************
['13', '14065', '5.', 'Lucas Papi', 'IT', 'Master 1 Hommes', '1980', '', Timedelta('4 days 05:21:47')] 
 ********************
['11', '14366', '6.', 'Brice Jacquot', 'FR', 'Master 1 Hommes', '1980', 'Hoka', Timedelta('4 days 06:14:19')] 
 ********************
[

Fetching from: https://my4.raceresult.com/161079/RRPublish/data/list?key=374250657b42ce1a7a8e1a4a3311786d&listname=Result%20Lists%7CProv.%20Gender%20Results%20360&page=results&contest=1&r=group&name=%231_Female&f=
['5', '12490', '1.', 'Anita Lehmann', 'CH', 'Master 1 Femmes', '1980', '', Timedelta('4 days 09:06:23')] 
 ********************
['7', '14420', '2.', 'Denise Zimmermann', 'CH', 'Master 1 Femmes', '1975', '', Timedelta('4 days 17:44:25')] 
 ********************
['102', '12586', '3.', 'Alessandra Olivi', 'IT', 'Seniors Femmes', '1991', '', Timedelta('4 days 20:03:19')] 
 ********************
['367', '14120', '4.', 'Katja Fink', 'CH', 'Master 1 Femmes', '1975', '', Timedelta('4 days 20:36:47')] 
 ********************
['91', '12573', '5.', 'Héloïse Aeberli', 'FR', 'Seniors Femmes', '1987', '', Timedelta('5 days 00:23:27')] 
 ********************
['376', '14379', '6.', 'Aneta Ralcheva', 'BG', 'Master 2 Femmes', '1967', '', Timedelta('5 days 04:13:29')] 
 ********************
['235'

In [19]:
SP_dem_df =  pd.DataFrame(runner_list, columns = ['Bib', 'ID', 'Status', 'Name', 
                               'Nationality',
                               'Age Category',
                                'Birth Year', 'Club', 'Duration'])   
SP_dem_df['Status'] = np.where(SP_dem_df['Status'] == 'DNF', 'DNF', 'Finished')
SP_dem_df['Year'] = year
SP_dem_df['Race'] = 'SP360'

SP_dem_df.loc[SP_dem_df['Age Category'].str.contains('Hommes'), 'Gender'] = 'M'
SP_dem_df.loc[SP_dem_df['Age Category'].str.contains('Femmes'), 'Gender'] = 'F'
SP_dem_df['Age Category'] = SP_dem_df['Age Category'].str.replace(' Femmes', '')
SP_dem_df['Age Category'] = SP_dem_df['Age Category'].str.replace(' Hommes', '')

SP_dem_df['PK'] = SP_dem_df['Race'] +'_' + SP_dem_df['Year'] +'_' + SP_dem_df['Bib']    

SP_dem_df = SP_dem_df[[
     'Bib', 'ID','Name','Status', 'Nationality', 'Gender', 
    'Birth Year', 'Duration', 'Year', 'Race', 'PK']]

SP_dem_df.head()

,Bib,ID,Name,Status,Nationality,Gender,Birth Year,Duration,Year,Race,PK
0,2,14362,Peter Kienzl,Finished,IT,M,1975,3 days 14:34:28,2021,SP360,SP360_2021_2
1,4,12489,Michael Nançoz,Finished,CH,M,1988,3 days 21:51:22,2021,SP360,SP360_2021_4
2,369,14181,Hendrik Spoering,Finished,DE,M,1990,4 days 03:18:46,2021,SP360,SP360_2021_369
3,36,12514,Nicolas Bie,Finished,FR,M,1977,4 days 04:37:23,2021,SP360,SP360_2021_36
4,13,14065,Lucas Papi,Finished,IT,M,1980,4 days 05:21:47,2021,SP360,SP360_2021_13


In [20]:
SP_dem_df.to_excel(f'Data/SwissPeaks360_{year}_DEM.xlsx', index=False)

In [21]:
SP_dem_df[SP_dem_df['Status'] == 'DNF']

,Bib,ID,Name,Status,Nationality,Gender,Birth Year,Duration,Year,Race,PK
164,3,13380,Serge Lattion,DNF,CH,M,1973,NaT,2021,SP360,SP360_2021_3
165,6,14408,Joachim Pellissier,DNF,CH,M,1978,NaT,2021,SP360,SP360_2021_6
166,10,14488,Jack B Liver,DNF,CH,M,1986,NaT,2021,SP360,SP360_2021_10
167,16,12494,Didier Metral,DNF,FR,M,1979,NaT,2021,SP360,SP360_2021_16
168,19,12497,Carlos Galvache Oliver,DNF,ES,M,1974,NaT,2021,SP360,SP360_2021_19
...,...,...,...,...,...,...,...,...,...,...,...
317,176,12664,Stéphanie Perriard,DNF,CH,F,1985,NaT,2021,SP360,SP360_2021_176
318,178,12666,Satu Iho,DNF,FI,F,1984,NaT,2021,SP360,SP360_2021_178
319,316,13652,Myrianna De Icco,DNF,CH,F,1986,NaT,2021,SP360,SP360_2021_316
320,335,13781,Berengere Bastiani,DNF,FR,F,1969,NaT,2021,SP360,SP360_2021_335


### Using bib column to extract station 

In [6]:
list(SP_dem_df['ID'].unique())[:1]

['14362']

In [7]:
station_list = []
failed_to_fetch= []
n=0
# Loop through the unique bibs in the DataFrame
for unique_id in list(SP_dem_df['ID'].unique()):  # Limit to just one bib for testing
    n = n+1
    print(n, ' - ', unique_id)
    
    # URL of the JSON file
    url = f'https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid={unique_id}'
    print(url)

    # Send a GET request to fetch the JSON data
    response = requests.get(url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON data
        data = response.json()
        stations = data['Splits']
        
        # Loop through the stations
        for station in stations:
            station_name = station['Name']
            exists = station['Exists']
            
            # Helper function to format durations safely
            def get_formatted_duration(key):
                duration = station.get(key)
                if duration:
                    return f"{duration.split(':')[0]} days {':'.join(duration.split(':')[1:])}"
                return np.nan

            formatted_gun = get_formatted_duration('Gun')
            formatted_chip = get_formatted_duration('Chip')
            formatted_sector = get_formatted_duration('Sector')

            # Store the extracted info in the station list
            extracted_info = [unique_id, station_name, exists,
                              formatted_gun, 
                              formatted_chip,
                              formatted_sector]
            station_list.append(extracted_info)
            
    else:
        print(f"Failed to fetch data for {unique_id}. Status code: {response.status_code}")
        failed_to_fetch.append(unique_id)
        
    
    #### So we dont get blocked!!!!
    delay_seconds = random.randint(5,10)
    print('Delay before calling the server again: ', delay_seconds)
    print('*'*20)
    time.sleep(delay_seconds)



1  -  14362
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=14362
Delay before calling the server again:  10
********************
2  -  12489
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12489
Delay before calling the server again:  6
********************
3  -  14181
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=14181
Delay before calling the server again:  8
********************
4  -  12514
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12514
Delay before calling the server again:  7
********************
5  -  14065
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=14065
Delay before calling the server again:  8
********************
6  -  14366
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=

47  -  12708
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12708
Delay before calling the server again:  6
********************
48  -  12546
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12546
Delay before calling the server again:  8
********************
49  -  13362
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13362
Delay before calling the server again:  8
********************
50  -  12636
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12636
Delay before calling the server again:  6
********************
51  -  12531
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12531
Delay before calling the server again:  10
********************
52  -  12549
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786

93  -  13471
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13471
Delay before calling the server again:  8
********************
94  -  12653
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12653
Delay before calling the server again:  5
********************
95  -  13606
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13606
Delay before calling the server again:  6
********************
96  -  12620
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12620
Delay before calling the server again:  10
********************
97  -  12560
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12560
Delay before calling the server again:  8
********************
98  -  12515
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786

Delay before calling the server again:  9
********************
139  -  13409
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13409
Delay before calling the server again:  6
********************
140  -  12682
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12682
Delay before calling the server again:  10
********************
141  -  12541
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12541
Delay before calling the server again:  7
********************
142  -  12554
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12554
Delay before calling the server again:  8
********************
143  -  13954
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13954
Delay before calling the server again:  6
********************
144  -  13411
https://my4.raceresult

Delay before calling the server again:  6
********************
185  -  12536
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12536
Delay before calling the server again:  8
********************
186  -  12537
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12537
Delay before calling the server again:  10
********************
187  -  12542
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12542
Delay before calling the server again:  7
********************
188  -  12543
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12543
Delay before calling the server again:  9
********************
189  -  12545
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12545
Delay before calling the server again:  5
********************
190  -  12551
https://my4.raceresult

Delay before calling the server again:  10
********************
231  -  12690
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12690
Delay before calling the server again:  7
********************
232  -  12694
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12694
Delay before calling the server again:  5
********************
233  -  12695
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12695
Delay before calling the server again:  6
********************
234  -  12696
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12696
Delay before calling the server again:  10
********************
235  -  12698
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=12698
Delay before calling the server again:  8
********************
236  -  12703
https://my4.raceresul

Delay before calling the server again:  9
********************
277  -  13774
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13774
Delay before calling the server again:  7
********************
278  -  13779
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13779
Delay before calling the server again:  9
********************
279  -  13785
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13785
Delay before calling the server again:  5
********************
280  -  13799
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13799
Delay before calling the server again:  8
********************
281  -  13813
https://my4.raceresult.com/161079/RRPublish/data/splits?key=374250657b42ce1a7a8e1a4a3311786d&pid=13813
Delay before calling the server again:  9
********************
282  -  13899
https://my4.raceresult.

Delay before calling the server again:  7
********************


In [14]:
# Convert the station list to a DataFrame
aid_station_df = pd.DataFrame(station_list, columns= ['ID', 'Aid Station', 'Exists?',
                              'formatted_gun', 
                              'formatted_chip',
                              'formatted_sector'])

aid_station_df['Year'] = year
aid_station_df['Race'] = 'SP360'

aid_station_df['PK'] = aid_station_df['Race'] +'_' + aid_station_df['Year'] +'_' + aid_station_df['ID']    

# Preview the DataFrame
print(aid_station_df.head())
# aid_station_df.to_excel(f'Data/SwissPeaks360_{year}_aid_stations.xlsx', index=False)

      ID Aid Station  Exists?  formatted_gun formatted_chip formatted_sector  \
0  14362  START 360K     True     00 days 03            NaN              NaN   
1  14362   Fiesch IN     True   6 days 53:08   6 days 53:05     6 days 53:05   
2  14362  Fiesch OUT     True   7 days 22:08   7 days 22:05       29 days 00   
3  14362   Eisten IN     True  17 days 23:43  17 days 23:41    10 days 01:36   
4  14362  Eisten OUT     True  17 days 50:47  17 days 50:45       27 days 04   

   Year   Race                PK  
0  2021  SP360  SP360_2021_14362  
1  2021  SP360  SP360_2021_14362  
2  2021  SP360  SP360_2021_14362  
3  2021  SP360  SP360_2021_14362  
4  2021  SP360  SP360_2021_14362  


In [15]:
aid_station_df['Aid Station'].unique()

array(['START 360K', 'Fiesch IN', 'Fiesch OUT', 'Eisten IN', 'Eisten OUT',
       'Grimentz IN', 'Grimentz OUT', 'Grande-Dixence IN',
       'Grande-Dixence OUT', 'Finhaut IN', 'Finhaut OUT', 'Crosets IN',
       'Crosets OUT', 'Annonceur', 'FINISH Bouveret'], dtype=object)

In [16]:
aid_station_df.to_excel(f'Data/SwissPeaks360_{year}_aid_stations.xlsx', index=False)

In [13]:
# End the timer
end_time = time.time()

# Calculate and print execution time
execution_time = end_time - start_time
print(f"Execution time: {execution_time:.6f} seconds")

Execution time: 2744.359657 seconds
